# Task 2

In [1]:
!pip install -r requirements.txt

In [4]:
# import dependencies
import datasets
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import random
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from torch.cuda.amp import autocast
import os 

In [5]:
DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"
MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"

ext_data = pd.read_csv("./tasks/External-Dataset_for_Task2.csv")

In [5]:
########################################################
# Entry point
########################################################7

class SMILESDataset(Dataset):
    def __init__(self, strings, labels):
        self.strings = strings
        self.labels = labels

    def __len__(self):
        return len(self.strings)

    def __getitem__(self, idx):
        string = self.strings[idx]
        target = self.labels[idx]
        return [string, target]

class MoLFormerWithRegressionHead(nn.Module):
    def __init__(self, model, hidden_size=768):
        super(MoLFormerWithRegressionHead, self).__init__()
        self.encoder = model

        # regression head (fully connected layer)
        self.regressor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, 1)
        )
        
    def forward(self, inputs):
        outputs = self.encoder(**inputs)
        model_representation = outputs.pooler_output
        regression_output = self.regressor(model_representation)
        return regression_output

## Influence score calculation

In [6]:
# Function to compute gradients
def compute_gradients(model, tokenizer, smile_strings, smile_targets, loss_fn):
    """Compute gradients of the loss with respect to model parameters."""
    model.eval()

    inputs = tokenizer(smile_strings, padding=True, return_tensors="pt").to(device)
    
    model.zero_grad()
    outputs = model(inputs)

    loss = loss_fn(outputs, smile_targets)
    loss.backward()
    
    grad = torch.cat([param.grad.detach().view(-1) for param in model.parameters()])
    return grad

def lissa_approximation(model, tokenizer, train_loader, num_iters=100, damping=0.01):
    model.eval()

    cur_estimate = None  # Match first gradient shape
    inv_hvp = None  # Store final inverse Hessian estimate

    for iter_idx in tqdm(range(num_iters), desc="Estimating iHVP"):
        for batch_idx, batch in enumerate(train_loader):
            smile_strings, smile_targets = batch
            smile_targets = smile_targets.to(device).float()

            inputs = tokenizer(smile_strings, padding=True, return_tensors="pt").to(device)
        
            model.zero_grad()
            outputs = model(inputs)
            loss = torch.nn.functional.mse_loss(outputs, smile_targets)

            grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True)
            grads = torch.cat([g.detach().view(-1) for g in grads])
            

            # Print gradient shapes for the first batch
            if iter_idx == 0 and batch_idx == 0:
                print(f" Gradient Shape: {grads.shape}")
                cur_estimate = torch.zeros_like(grads)  # Match first gradient shape
                inv_hvp = torch.zeros_like(grads) 
                print(f" Initial cur_estimate shape: {cur_estimate.shape}")

            cur_estimate -= damping * grads
        inv_hvp += cur_estimate  # Accumulate estimates

    inv_hvp /= num_iters  # Normalize by iteration count to prevent oversized values
    print(f"** Final iHVP shape: {inv_hvp.shape}")
    return inv_hvp


def compute_influence_scores(model, tokenizer, ext_loader, test_loader, loss_fn):
    
    # Compute inverse Hessian-vector product (iHVP)
    iHVP = lissa_approximation(model, tokenizer, test_loader)

    print(f"Final iHVP Shape: {iHVP.shape}")

    # Compute influence scores
    influence_scores = []

    for batch_idx, batch in enumerate(ext_loader):
        smile_strings, smile_targets = batch
        smile_targets = smile_targets.to(device).float()
        ext_grad = compute_gradients(model, tokenizer, smile_strings, smile_targets, loss_fn)
        # calculating the influence of single data point of the external data on test set
        score = torch.dot(ext_grad.flatten(), iHVP)  
        influence_scores.append((batch_idx, score.item()))
        
    return influence_scores

def get_top_k(influence_scores, top_k):
    # Convert to DataFrame
    influence_df = pd.DataFrame(influence_scores, columns=["Index", "Influence Score"])

    # Add Influence Score column to ext_data and save it
    ext_data["Influence Score"] = None  # Initialize column
    ext_data.loc[influence_df["Index"], "Influence Score"] = influence_df["Influence Score"].values
    ext_data.to_csv("External_Data_with_influence_scores.csv", index=False)
    
    # Keep only negative influence scores
    influence_df = influence_df[influence_df["Influence Score"] > 0]
    print(f"Number of positive samples: {len(influence_df)}")

    # Sort by influence score (descending)
    influence_df = influence_df.sort_values(by="Influence Score", ascending=False)

    # Select the top influential samples
    top_indices = influence_df.nlargest(top_k, "Influence Score")["Index"]
    selected_data = ext_data.loc[top_indices]

    # Save selected data for fine-tuning
    selected_data.to_csv("Selected_External_Data.csv", index=False)
    
    top_k_indices = top_indices.tolist()
    return top_k_indices

## Training and Testing

In [7]:
# training
def train(train_dataloader, model, tokenizer, epochs, loss_fn, optimizer, save_path):
    model.train()
    best_loss = 1000
    for epoch in range(epochs):
        running_loss = 0
        count = 0
        with tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for index, data in enumerate(pbar):
                smile_strings = data[0]
                smile_targets = data[1].to(device).float()

                inputs = tokenizer(smile_strings, padding=True, return_tensors="pt").to(device)
                outputs = model(inputs)

                loss = loss_fn(outputs, smile_targets)

                running_loss = loss.item() + running_loss
                count = count + len(data)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        epoch_loss = running_loss / count
        print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
        
        # saving the model best model
        if epoch_loss < best_loss :
            best_loss = epoch_loss
            torch.save(model.state_dict(), save_path)


In [8]:
# testing 
def test(test_dataloader, model):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        with tqdm(test_dataloader, desc="Testing", unit="batch") as pbar:
            for data in pbar:
                smile_strings = data[0]
                smile_targets = data[1].to(device).float()

                inputs = tokenizer(smile_strings, padding=True, return_tensors="pt").to(device)
                outputs = model(inputs)  # Flatten output to match targets

                predictions.extend(outputs.cpu().numpy())
                actuals.extend(smile_targets.cpu().numpy())

    # Convert lists to numpy arrays
    predictions = np.array(predictions).flatten()
    actuals = np.array(actuals).flatten()
    
    # Ensure correct dtype
    predictions = predictions.astype(np.float64)
    actuals = actuals.astype(np.float64)

    predictions = predictions.reshape(-1, 1)
    actuals = actuals.reshape(-1, 1)

    # Scale back
    predictions = np.array(scaler.inverse_transform(predictions)).flatten().tolist()
    actuals = np.array(scaler.inverse_transform(actuals)).flatten().tolist()

    # Calculate metrics
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    pearson_corr, _ = pearsonr(actuals, predictions)
    spearman_corr, _ = spearmanr(actuals, predictions)

    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Pearson Correlation: {pearson_corr:.4f}")
    print(f"Spearman Correlation: {spearman_corr:.4f}")

## Running the task 2

In [10]:
# setting the seed

seed = 100
# Masked LM which is domain adapted with regression head from Task 1
model_save_path = "unsupervised_model.pth"

# for reproducibilty
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# downloading the dataset fro huggigface
lipophilicity_df = pd.read_csv("hf://datasets/scikit-fingerprints/MoleculeNet_Lipophilicity/lipophilicity.csv")

# information regarding the dataset
print(lipophilicity_df.shape)
print(lipophilicity_df.info())
print(lipophilicity_df.head())
lipophilicity_strings = np.array(lipophilicity_df['SMILES'].values)
lipophilicity_targets = np.array(lipophilicity_df['label'].values)

# scaling the target values between (0,1)
scaler = MinMaxScaler()
lipophilicity_targets = scaler.fit_transform(lipophilicity_targets.reshape(-1,1))
X_train, X_test, y_train_scaled, y_test_scaled = train_test_split(lipophilicity_strings, lipophilicity_targets, test_size = 0.2, random_state=seed)

#ext_data = pd.read_csv("External-Dataset_for_Task2.csv")

# scaling the target values between (0,1) of external data
ext_data_strings = ext_data['SMILES'].values
ext_data_targets = np.array(ext_data['Label'].values)
y_ext_scaled = scaler.fit_transform(ext_data_targets.reshape(-1,1))

print("data reading complete")

# creating external dataloader
ext_dataset = SMILESDataset(ext_data_strings, y_ext_scaled)
ext_dataloader = DataLoader(ext_dataset, batch_size=1, shuffle=False)

# creating test dataloader
test_dataset = SMILESDataset(X_test, y_test_scaled)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# loading the tokenizer and model 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(MODEL_NAME, deterministic_eval=True, trust_remote_code=True)
regression_model = MoLFormerWithRegressionHead(model).to(device)
regression_model.load_state_dict(torch.load(model_save_path, map_location=device))

# decalring loss
mse_loss = torch.nn.MSELoss()
influences = compute_influence_scores(regression_model, tokenizer, ext_dataloader, test_dataloader, mse_loss)
print(influences)
top_k_indices = get_top_k(influences, top_k=20)
top_k = 20
print(f"Top {top_k} positive influential training indices")

# taking a subset of the external data with top_k highest influence scores
ext_dataset = Subset(ext_dataset, top_k_indices)
train_dataset = SMILESDataset(X_train, y_train_scaled)
combined_dataset = torch.utils.data.ConcatDataset([ext_dataset, train_dataset])
combined_dataloader = DataLoader(combined_dataset, batch_size=16, shuffle=True)

# loading the model trained in Task 1 and creating new model with regression head
model_path = "./molformer_finetuned"
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regression_model = MoLFormerWithRegressionHead(model).to(device)

epochs = 8
learning_rate = 0.0001
model_save_path = "combined_model.pth"
mse_loss = nn.MSELoss()
optimizer = optim.Adam(regression_model.parameters(), lr=learning_rate)

# training
train(
    combined_dataloader,
    regression_model,
    tokenizer,
    epochs = epochs,
    loss_fn=mse_loss,
    optimizer=optimizer,
    save_path=model_save_path
)

# testing
regression_model = MoLFormerWithRegressionHead(model).to(device)
regression_model.load_state_dict(torch.load(model_save_path, map_location=device))
test(
    test_dataloader,
    regression_model
)